## Crunching the numbers on Vermont colleges

An analysis of federal higher ed data via the IPEDS data center.

Source for all these numbers: https://nces.ed.gov/ipeds/use-the-data 
Go to: survey data > complete data files > use dropdown for year/type of data.

_Tip: Download and read the data dictionary for every data file you download. Watch out for changes in column names and definitions from year to year. It will save you many headaches._

### First thing's first: What schools are we looking for?

IPEDS data uses unique ID numbers rather than names for every school, so before we do anything else, we need to figure out which ID numbers we're looking for. Luckily, the "institutional characteristics" files contain an ID column, a school name and a state column, so we can filter on that.

We're interested in school data since 2000, but since the list of reporting institutions don't change that much from year to year, I'm just going to take a sampling of annual files to make sure I'm getting everything.

In [1]:
import pandas as pd
import numpy as np

In [2]:
school_files = [
    "raw_data/fa2000hd.csv","raw_data/hd2012.csv", "raw_data/hd2016.csv"
]

# we'll add everything to this empty dataframe
vt_schools = pd.DataFrame()

for i in school_files:
    # there's funky encoding in one of the files, so read in with 
    # latin-1 encoding.
    school = pd.read_csv(i,encoding="latin-1", low_memory=False)
    
    # grab the year from the filename so we can grab only the most recent row.
    year=i[14:18]
    school["year"] = year
    
    if year > "2001":
        school = school[school["STABBR"] == "VT"]
        school = school[["UNITID","INSTNM", "year"]]
        vt_schools = vt_schools.append(
            school,ignore_index=True, sort=False
        )
    else:
        school = school[school["stabbr"] == "VT"]
        school.rename(columns={
            "unitid":"UNITID",
            "instnm":"INSTNM"
        }, inplace=True)
        vt_schools = vt_schools.append(
            school[["UNITID", "INSTNM","year"]],
            ignore_index=True,
            sort=False
        )
        
vt_schools.rename(columns={
    "INSTNM":"school"
}, inplace=True)

# keep only the most recent version of the school's name
vt_schools = vt_schools.sort_values(
        "year"
    ).drop_duplicates(
        subset=["UNITID"], keep="last"
    )[["UNITID", "school"]]

# We're interested only in colleges, so we're going to drop some of these.
notacollege = [444459,250832,231004,456001,230843]
vt_schools = vt_schools[
        ~vt_schools["UNITID"].isin(notacollege)
    ].reset_index(
        drop=True
    )
vt_schools["school"] = vt_schools["school"].str.title()

# Eh, title-casing doesn't work for every instance, so 
# I'm gonna be neurotic and tweak some stuff.
vt_schools["school"] = vt_schools["school"].replace(
    {
        "Saint Michael'S College":"Saint Michael's College",
        "Of": "of",
        "Sit ":"SIT ",
        "-office":"-Office"
    }, regex=True
)

vt_schools.head()

,UNITID,school
0,431804,Rutland Regional Medical Center School of Rad ...
1,231040,Rutland Regional Medical Center
2,231192,Woodbury College
3,377494,Fletcher Allen Health Care School of Cytotech
4,382027,New England Culinary Institute At Essex


Not all of these schools are still in existence, and we're going to want to focus in on them in some cases. So let's also throw in a boolean column that we can reference in the future.

*Note: We're including Lyndon and Johnson colleges in our shortlist, even though they merged to form Northern Vermont University in July 2018.*

In [3]:
doesnotexist = [431804,231040,231192,377494,382027,231110,230825]

vt_schools["exists"] = ~vt_schools["UNITID"].isin(doesnotexist)

vt_schools.to_csv("data/vt_college_list.csv")
vt_schools.head()

,UNITID,school,exists
0,431804,Rutland Regional Medical Center School of Rad ...,False
1,231040,Rutland Regional Medical Center,False
2,231192,Woodbury College,False
3,377494,Fletcher Allen Health Care School of Cytotech,False
4,382027,New England Culinary Institute At Essex,False


### Let's check out enrollment

Now we're ready to jump into the data. Let's start by pulling fall enrollment numbers since 2000 for all of our colleges.

Pay attention to the "unique on" notes in these data dictionaries. Each institution has multiple lines that break out subsets of the population, so unless you filter out everything but the totals row, you're going to end up with duplicates.

In [4]:
enrollment_files = [
    "raw_data/ef2000a.csv","raw_data/ef2001a.csv", 
    "raw_data/ef2002a.csv","raw_data/ef2003a.csv",
    "raw_data/ef2004a.csv","raw_data/ef2005a.csv",
    "raw_data/ef2006a_RV.csv", "raw_data/ef2007a_rv.csv",
    "raw_data/ef2008a_rv.csv","raw_data/ef2009a_rv.csv",
    "raw_data/ef2010a_rv.csv","raw_data/ef2011a_rv.csv",
    "raw_data/ef2012a_rv.csv","raw_data/ef2013a_rv.csv",
    "raw_data/ef2014a_rv.csv","raw_data/ef2015a_rv.csv",
    "raw_data/ef2016a.csv"
]

vt_enrollment = pd.DataFrame()

for i in enrollment_files:
    enroll = pd.read_csv(i)
    
    # grab the year from the filename
    year=i[14:18]
    enroll["year"] = year
        
    # in pre-2006 files, there's no "grand total" column.
    if year<"2006":
        enroll.rename(columns={
            "unitid":"UNITID",
            "lstudy":"level",
            "efrace15":"men",
            "efrace16":"women"
        },inplace=True)
        enroll = enroll[enroll["line"] == 29]
        enroll["enrollment"] = enroll["men"] + enroll["women"]
    
    # column names and meanings change starting in 2006.
    elif year < "2008":
        enroll.rename(columns={
            "EFRACE24":"enrollment",
            "EFALEVEL":"level"
        },inplace=True)
        enroll = enroll[enroll["level"] == 1]
        
    # and again starting in 2008, ugh.
    else:
        enroll.rename(columns={
            "EFTOTLT":"enrollment",
            "EFALEVEL":"level"
        }, inplace=True)
    
        enroll = enroll[enroll["level"] == 1]
    
    # drop any row that's not in our `vt_schools` dataframe
    enroll = pd.merge(
        vt_schools, enroll[["UNITID","enrollment","year"]], 
        how='inner', right_on="UNITID", left_on="UNITID"
    )
        
    # now append it to our big enrollment dataframe
    vt_enrollment = vt_enrollment.append(
        enroll, ignore_index=True, sort=False
    )

vt_enrollment.to_csv("data/vt_enrollment.csv", index=False)
vt_enrollment.head()

,UNITID,school,exists,enrollment,year
0,431804,Rutland Regional Medical Center School of Rad ...,False,5,2000
1,231192,Woodbury College,False,149,2000
2,382027,New England Culinary Institute At Essex,False,160,2000
3,230825,Burlington College,False,206,2000
4,231068,SIT Graduate Institute,True,490,2000


Cool. Let's also grab some statewide numbers just so we have 'em.

In [5]:
total_enrollment = vt_enrollment[["enrollment","year"]].groupby("year").sum()

total_enrollment.to_csv("data/vt_total_enrollment.csv")
total_enrollment.head()

,enrollment
year,
2000,35494
2001,36364
2002,36547
2003,37846
2004,38662


Lastly, let's grab percent change in enrollment for each school over the past 10 years.

In [6]:
years = ["2007", "2016"]
enrollment_change = vt_enrollment[vt_enrollment["year"].isin(years)]
enrollment_change = enrollment_change.pivot_table(
    index=["UNITID","school"], columns="year",
    values="enrollment"
).reset_index().dropna(subset=["2007","2016"])
enrollment_change["change"] = (
        enrollment_change["2016"]-enrollment_change["2007"]
    )/enrollment_change["2007"]

enrollment_change.to_csv("data/vt_enrollment_change.csv", index=False)
enrollment_change.head()

year,UNITID,school,2007,2016,change
0,230816,Bennington College,723.0,805.0,0.113416
2,230834,Castleton University,2144.0,2342.0,0.092351
3,230852,Champlain College,2796.0,4778.0,0.708870
4,230861,Community College of Vermont,5608.0,5863.0,0.045471
5,230889,Goddard College,678.0,505.0,-0.255162


### Next up: admissions!

This is another one where the variables we're looking for change partway through. Oh, and fun times: the 2009 through 2013 datasets use a period for blank cells. They don't make it easy.

Since the naming convention is different in the earlier files, we're going to pull these in through three separate functions.

Let's go in order of age, oldest first.

In [7]:
vt_adm = pd.DataFrame()

#process oldest files
adm_older_files = ["raw_data/ic2007.csv","raw_data/ic2008_rv.csv"]

for i in adm_older_files:
    coll = pd.read_csv(i)
    year = i[14:18]
    coll["year"] = year
    
    coll.rename(columns={
        "APPLCNM":"applications-men",
        "APPLCNW":"applications-women",
        "ADMSSNM":"admissions-men",
        "ADMSSNW":"admissions-women",
    }, inplace=True)
    coll = coll.replace(".", np.nan)
    
    coll["admissions-men"] = pd.to_numeric(coll["admissions-men"])
    coll["admissions-women"] = pd.to_numeric(coll["admissions-women"])
    coll["applications-men"] = pd.to_numeric(coll["applications-men"])
    coll["applications-women"] = pd.to_numeric(coll["applications-women"])
    
    coll["applications"] = coll["applications-men"] + coll["applications-women"]
    coll["admissions"] = coll["admissions-men"] + coll["admissions-women"]

    coll = pd.merge(
        vt_schools, coll, how='inner',on="UNITID"
    )
    vt_adm = vt_adm.append(coll, ignore_index=True, sort=False)
    
#process medium-old files
adm_old_files = [
    "raw_data/ic2009_rv.csv","raw_data/ic2010_rv.csv",
    "raw_data/ic2011_rv.csv","raw_data/ic2012_rv.csv",
    "raw_data/ic2013_rv.csv"
]

for i in adm_old_files:
    coll = pd.read_csv(i)
    year=i[14:18]
    coll["year"] = year
    
    coll.rename(columns={
        "ADMSSN":"admissions",
        "APPLCN":"applications",
    }, inplace=True)
    
    coll = pd.merge(
        vt_schools, coll, how='inner',on="UNITID"
    )
    
    coll = coll.replace(".", np.nan)
    coll["admissions"] = pd.to_numeric(coll["admissions"])
    coll["applications"] = pd.to_numeric(coll["applications"])
    
    vt_adm = vt_adm.append(coll, ignore_index=True, sort=False)

#process newer files
adm_new_files = [
    "raw_data/adm2014_rv.csv","raw_data/adm2015_rv.csv",
    "raw_data/adm2016.csv"
]

for i in adm_new_files:
    coll = pd.read_csv(i)
    year = i[15:19]
    coll["year"] = year
    
    coll.rename(columns={
        "APPLCN":"applications",
        "ADMSSN":"admissions",
    }, inplace=True)
    
    coll = pd.merge(
        vt_schools, coll, how='inner',on="UNITID"
    )
    vt_adm = vt_adm.append(coll, ignore_index=True, sort=False)

vt_adm.head()

,UNITID,school,exists,PEO1ISTR,PEO2ISTR,PEO3ISTR,PEO4ISTR,PEO5ISTR,PEO6ISTR,CNTLAFFI,...,DISTNCED,FTGDNIDP,PTGDNIDP,DOCPP,DOCPPSP,DSTNCED1,DSTNCED2,DSTNCED3,ACTWR75,ACTMT75
0,231192,Woodbury College,False,1.0,1.0,0.0,0.0,0.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,382027,New England Culinary Institute At Essex,False,1.0,1.0,0.0,0.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,230825,Burlington College,False,1.0,1.0,0.0,0.0,0.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,231068,SIT Graduate Institute,True,0.0,1.0,1.0,0.0,0.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,231077,College of St Joseph,True,0.0,1.0,0.0,0.0,0.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Whoa. Now that we've got a crazy file with many columns, we're going to narrow it down and do some calculations.

In [8]:
vt_adm = vt_adm[["UNITID","school","applications","admissions","year","exists"]]

vt_adm["admit_rate"] = vt_adm["admissions"]/vt_adm["applications"]
vt_adm = vt_adm.dropna(subset=["applications","admissions"])

vt_adm.to_csv("data/vt_admissions.csv", index=False)
vt_adm.head()

,UNITID,school,applications,admissions,year,exists,admit_rate
0,231192,Woodbury College,23.0,22.0,2007,False,0.956522
2,230825,Burlington College,69.0,43.0,2007,False,0.623188
4,231077,College of St Joseph,101.0,89.0,2007,True,0.881188
5,231086,Southern Vermont College,344.0,316.0,2007,True,0.918605
6,231095,Sterling College,97.0,74.0,2007,True,0.762887


### First-year undergrads by region

One more easy(ish) one, and then we'll jump into the finance data. So, you know, get excited.

How many students are coming to each Vermont from out of state? Out of the country? Schools report that to IPEDS, but a couple notes on that:
- These numbers are _only for first-year, first-time degree-seeking undergrads_.
- Schools only have to report them _every other year_, so be aware of that if you're trying to make year-to-year comparisons.

Again, **READ THAT DATA DICTIONARY**.

In this case, we're just going to look at the most recent reported year, 2016.

In [14]:
vt_geo = pd.read_csv("raw_data/ef2016c.csv")

vt_geo.rename(columns={
    "EFCSTATE":"state",
    "EFRES01":"students"
}, inplace=True)

vt_geo = pd.merge(
    vt_schools, vt_geo, how="inner", on="UNITID"
)[["UNITID","school","state","students"]]

# exclude rows for grand total and US total
excludecodes = [58, 99]
vt_geo = vt_geo[~vt_geo.state.isin(excludecodes)]

def region(code):
    if code == 50:
        return "in-state"
    elif code == 90:
        return "international"
    else:
        return "out-of-state"
    
vt_geo["region"] = vt_geo["state"].map(region)
vt_geo = vt_geo[["UNITID","school","students","region"]].groupby(
    ["UNITID","school","region"]
).sum().reset_index()

vt_geo.head()

,UNITID,school,region,students
0,230816,Bennington College,in-state,9
1,230816,Bennington College,international,35
2,230816,Bennington College,out-of-state,174
3,230834,Castleton University,in-state,315
4,230834,Castleton University,international,14


In [24]:
vt_geo_wide = vt_geo.pivot_table(
    index=["UNITID","school"], 
    columns="region",
    values="students"
).reset_index().fillna(0)

vt_geo_wide["total"] = vt_geo_wide["in-state"] + \
                       vt_geo_wide["international"] + \
                       vt_geo_wide["out-of-state"]

# vt_geo.to_csv("data/vt_student_geo.csv", index=False)
vt_geo_wide.head()

region,UNITID,school,in-state,international,out-of-state,total
0,230816,Bennington College,9.0,35.0,174.0,218.0
1,230834,Castleton University,315.0,14.0,147.0,476.0
2,230852,Champlain College,106.0,8.0,549.0,663.0
3,230861,Community College of Vermont,522.0,3.0,35.0,560.0
4,230889,Goddard College,1.0,2.0,6.0,9.0


In [26]:
vt_geo = vt_geo_wide.melt(
    id_vars=["UNITID", "school", "total"],
    var_name="region",
    value_name = "students"
)

vt_geo["percent"] = vt_geo["students"]/vt_geo["total"]
vt_geo = vt_geo.drop("total", axis=1)

vt_geo.to_csv("data/vt_student_geo.csv", index=False)
vt_geo.head()

,UNITID,school,region,students,percent
0,230816,Bennington College,in-state,9.0,0.041284
1,230834,Castleton University,in-state,315.0,0.661765
2,230852,Champlain College,in-state,106.0,0.159879
3,230861,Community College of Vermont,in-state,522.0,0.932143
4,230889,Goddard College,in-state,1.0,0.111111


### Finance, woooo!

Okay, we've been putting it off for long enough. It's time to jump into college finances.

This is sorta complicated, because it turns out there are three different accounting systems depending on type of school — public, private nonprofit and private for-profit. We're going to grab top-line revenue and expense for each and combine them into one dataset.

Let's do public schools first.

Note here that the Vermont State Colleges report out enrollment, tuition and other data separately, but report some financial data under the umbrella of `Vermont State Colleges - Office of the Chancellor`. There's one revenue column that'll get us individual revenue and expense numbers for each school, and we're going to use that one.

In [20]:
pub_files = [
    "raw_data/f0910_f1a_rv.csv", "raw_data/f1011_f1a_rv.csv", "raw_data/f1112_f1a_rv.csv", 
    "raw_data/f1213_f1a_rv.csv", "raw_data/f1314_f1a_rv.csv", "raw_data/f1415_f1a_rv.csv", 
    "raw_data/f1516_f1a.csv",
]

vt_pub_coll = pd.DataFrame()
for i in pub_files:
    coll = pd.read_csv(i)
    year = i[13:15]
    coll["year"] = "20"+year
    
    coll = pd.merge(vt_schools, coll, how='inner',on="UNITID")
    vt_pub_coll = vt_pub_coll.append(coll, ignore_index=True, sort=False)

vt_pub_coll.rename(columns={
    "F1B01":"tuition_fees",
    "F1B25":"revenues",
    "F1C191":"expenses"
}, inplace=True)
vt_pub_coll = vt_pub_coll[[
    "UNITID", "school", "year", "tuition_fees", 
    "revenues", "expenses","exists"
]]
vt_pub_coll.head()

,UNITID,school,year,tuition_fees,revenues,expenses,exists
0,231165,Vermont Technical College,2009,15904752,38720138,39808142,True
1,231156,Vermont State Colleges-Office of The Chancellor,2009,0,2263831,2435429,True
2,231174,University of Vermont,2009,235504000,606166000,581900000,True
3,230931,Lyndon State College,2009,13678274,31273673,29862335,True
4,230913,Johnson State College,2009,15055152,32244997,32460078,True


In [21]:
priv_files = [
    "raw_data/f0809_f2_rv.csv", "raw_data/f0910_f2_rv.csv","raw_data/f1011_f2_rv.csv",
    "raw_data/f1112_f2_rv.csv","raw_data/f1213_f2_rv.csv", "raw_data/f1314_f2_rv.csv",
    "raw_data/f1415_f2_rv.csv","raw_data/f1516_f2.csv"
]

vt_priv_coll = pd.DataFrame()

for i in priv_files:
    coll = pd.read_csv(i)
    year = i[13:15]
    coll["year"] = "20" + year
    
    coll = pd.merge(vt_schools, coll, how='inner',on="UNITID")
    vt_priv_coll = vt_priv_coll.append(coll, ignore_index=True, sort=False)

vt_priv_coll.rename(columns={
    "F2D01":"tuition_fees",
    "F2D16":"revenues",
    "F2E131":"expenses",
}, inplace=True)

vt_priv_coll = vt_priv_coll[[
    "UNITID", "school", "year", "tuition_fees",
    "revenues","expenses","exists"
]]
vt_priv_coll.head()

,UNITID,school,year,tuition_fees,revenues,expenses,exists
0,230825,Burlington College,2008,2707575,3280605,3312041,False
1,231068,SIT Graduate Institute,2008,45703647,100112579,110696345,True
2,231077,College of St Joseph,2008,3574137,5059533,5217743,True
3,231086,Southern Vermont College,2008,5477954,9075000,9233912,True
4,231095,Sterling College,2008,1477151,3696866,4037075,True


In [22]:
forprofit_files = [
    "raw_data/f0809_f3_rv.csv", "raw_data/f0910_f3_rv.csv","raw_data/f1011_f3_rv.csv",
    "raw_data/f1112_f3_rv.csv","raw_data/f1213_f3_rv.csv", "raw_data/f1314_f3_rv.csv",
    "raw_data/f1415_f3_rv.csv","raw_data/f1516_f3.csv"
]

vt_forprofit_coll = pd.DataFrame()
for i in forprofit_files:
    coll = pd.read_csv(i)
    year = i[13:15]
    coll["year"] = "20"+year
    
    coll.rename(columns={
        "F3D01": "tuition_fees",
        "F3D09": "revenues",
    }, inplace=True)
    
    if year < "2013":
        coll.rename(columns={
            "F3B02": "expenses"
        }, inplace=True)
    else:
        coll.rename(columns={
            "F3E071": "expenses"
        }, inplace=True)
    
    coll = pd.merge(vt_schools, coll, how='inner',on="UNITID")
    vt_forprofit_coll = vt_forprofit_coll.append(
        coll, ignore_index=True, sort=False
    )

vt_forprofit_coll = vt_forprofit_coll[[
    "UNITID","school", "year","tuition_fees",
    "revenues","expenses","exists"
]]
    
vt_forprofit_coll.head()

,UNITID,school,year,tuition_fees,revenues,expenses,exists
0,230977,New England Culinary Institute,2008,14403049,21459014,24079155.0,True
1,230977,New England Culinary Institute,2009,13373011,18225068,17126490.0,True
2,230977,New England Culinary Institute,2010,12019083,15643259,16241955.0,True
3,230977,New England Culinary Institute,2011,11938399,15413400,16288105.0,True
4,230977,New England Culinary Institute,2012,9273622,12694283,13159095.0,True


In [23]:
vt_finance = pd.concat(
    [vt_priv_coll, vt_pub_coll,vt_forprofit_coll],
    join="outer",
    sort=True
)

vt_finance.to_csv("data/vt_finance.csv", index=False)
vt_finance.head()

,UNITID,exists,expenses,revenues,school,tuition_fees,year
0,230825,False,3312041.0,3280605,Burlington College,2707575,2008
1,231068,True,110696345.0,100112579,SIT Graduate Institute,45703647,2008
2,231077,True,5217743.0,5059533,College of St Joseph,3574137,2008
3,231086,True,9233912.0,9075000,Southern Vermont College,5477954,2008
4,231095,True,4037075.0,3696866,Sterling College,1477151,2008


Cool, cool, cool. We've got lots of stuff to work with. Let's move over to a new notebook and see what we can see; open the `02_explore` file next.